# Multi Agent STM: Strands with AgentCore Memory Tool

## Introduction

This notebook demonstrates how to implement a **multi-agent system with shared memory** using AWS AgentCore Memory and the Strands framework. While our previous examples focused on single-agent memory, this notebook explores how multiple specialized agents can work together while accessing a common memory store.

## What you will learn

- How to set up a shared memory resource that multiple agents can access
- Creating specialized agents as tools with their own memory access
- Implementing a coordinator agent that delegates to specialized agents
- Maintaining conversation context across multiple agent interactions

## Scenario context

In this example, we'll create a **Travel Planning System** with:
1. A Flight Booking Assistant specialized in air travel
2. A Hotel Booking Assistant focused on accommodations
3. A Travel Coordinator that delegates to these specialized agents

This approach demonstrates how complex domains can be broken down into specialized agents that share memory the same memory store.

## Architecture

![architecture](architecture.png)

## Prerequisites
- AWS account with appropriate permissions
- AWS IAM role with appropriate permissions for AgentCore Memory
- Access to Amazon Bedrock models

Let's get started by setting up our environment and creating our shared memory resource!

## Environment set up
Let's begin importing all the necessary libraries and defining the clients to make this notebook work.

In [6]:
!pip install -qr requirements.txt

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
awscli 1.40.37 requires botocore==1.38.38, but you have botocore 1.39.5 which is incompatible.
fastapi 0.115.13 requires starlette<0.47.0,>=0.40.0, but you have starlette 0.47.1 which is incompatible.
safety-schemas 0.0.14 requires pydantic<2.10.0,>=2.6.0, but you have pydantic 2.11.7 which is incompatible.
sagemaker 2.247.0 requires importlib-metadata<7.0,>=1.4.0, but you have importlib-metadata 8.7.0 which is incompatible.
sagemaker 2.247.0 requires packaging<25,>=23.0, but you have packaging 25.0 which is incompatible.
sagemaker-core 1.0.38 requires importlib-metadata<7.0,>=1.4.0, but you have importlib-metadata 8.7.0 which is incompatible.
sparkmagic 0.21.0 requires pandas<2.0.0,>=0.17.1, but you have pandas 2.2.3 which is incompatible.
sphinx 8.1.3 requires docutils<0.22,>=0.20, but you have docutils 0.19 whi

In [7]:
import os
import sys
import time
import boto3
import logging
import time
from datetime import datetime
from typing import Dict, List
from strands.hooks.registry import HookProvider, HookRegistry
from strands.hooks.events import MessageAddedEvent, AfterInvocationEvent, AgentInitializedEvent

Define the region and the role with the appropiate permissions for Amazon Bedrock models and AgentCore

In [8]:
region = "us-west-2"
role_arn = "<<INSERT-YOUR-IAM-ROLE>>"
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s", datefmt="%Y-%m-%d %H:%M:%S")
logger = logging.getLogger("agentcore-memory")

## Creating Shared Memory
In this section, we'll create a memory resource that will be shared among our specialized agents.

In [9]:
from BedrockAgentcore.memory import MemoryClient

In [10]:
client = MemoryClient(region_name=region)
memory_name = "TravelAgent_STM_%s" % datetime.now().strftime("%Y%m%d%H%M%S")
memory_id = None

2025-07-14 07:45:38 - INFO - Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole
2025-07-14 07:45:38 - INFO - Initialized MemoryClient for prod in us-west-2


In [11]:
try:
    print("Creating Memory...")
    memory_name = memory_name

    # Create the memory resource
    memory = client.create_memory_and_wait(
        name=memory_name,                       # Unique name for this memory store
        description="Travel Agent STM", # Human-readable description
        strategies=[],                          # No special memory strategies for short-term memory
        event_expiry_days=7,                    # Memories expire after 7 days
        memory_execution_role_arn=role_arn,     # IAM role with permissions to manage memory
        max_wait=300,                           # Maximum time to wait for memory creation (5 minutes)
        poll_interval=10                        # Check status every 10 seconds
    )

    # Extract and print the memory ID
    memory_id = memory['memoryId']
    print(f"Memory created successfully with ID: {memory_id}")
    
except Exception as e:
    # Handle any errors during memory creation
    print(f"❌ ERROR: {e}")
    import traceback
    traceback.print_exc()

    # Cleanup on error - delete the memory if it was partially created
    if memory_id:
        try:
            client.gmcp_client.delete_memory(memoryId=memory_id)
            print(f"Cleaned up memory: {memory_id}")
        except Exception as cleanup_error:
            print(f"Failed to clean up memory: {cleanup_error}")

Creating Memory...


2025-07-14 07:45:49 - INFO - Created memory: TravelAgent_STM_20250714074538-TAnbKI4KoR
2025-07-14 07:45:49 - INFO - Created memory TravelAgent_STM_20250714074538-TAnbKI4KoR, waiting for ACTIVE status...
2025-07-14 07:45:49 - INFO - Memory TravelAgent_STM_20250714074538-TAnbKI4KoR is now ACTIVE (took 0 seconds)


Memory created successfully with ID: TravelAgent_STM_20250714074538-TAnbKI4KoR


### Understanding Shared Memory for Multi-Agent Systems

The memory resource we've created will serve as a shared knowledge base for our travel planning system. All agents will read from and write to this common memory store, enabling:

1. **Knowledge Consistency**: All agents work with the same information
2. **Context Preservation**: Conversation history is maintained across agent transitions
3. **Specialized Access**: Each agent will have its own actor_id but share the session_id

This approach allows specialized agents to focus on their domains while still benefiting from the full conversation context.

### Step 3: Create Memory Hook Provider

This step defines our custom `MemoryHookProvider` class that automates memory operations. Hooks are special functions that run at specific points in an agent's execution lifecycle. The memory hook we're creating serves two primary functions:

1. **Retrieve Memories**: Automatically fetches relevant past conversations when a user sends a message
2. **Save Memories**: Stores new conversations after the agent responds

This creates a seamless memory experience without manual management.

In [39]:
class ShortTermMemoryHook(HookProvider):
    def __init__(self, memory_client: MemoryClient, memory_id: str, actor_id: str, session_id: str):
        self.memory_client = memory_client
        self.memory_id = memory_id
        self.actor_id = actor_id
        self.session_id = session_id
    
    def on_agent_initialized(self, event: AgentInitializedEvent):
        """Load recent conversation history when agent starts"""
        try:
            # Get last 5 conversation turns
            recent_turns = self.memory_client.get_last_k_turns(
                memory_id=self.memory_id,
                actor_id=self.actor_id,
                session_id=self.session_id,
                k=5,
                branch_name="main"
            )
            
            if recent_turns:
                # Format conversation history for context
                context_messages = []
                for turn in recent_turns:
                    for message in turn:
                        role = message['role'].lower()
                        content = message['content']['text']
                        context_messages.append(f"{role.title()}: {content}")
                
                context = "\n".join(context_messages)
                logger.info(f"Context from memory: {context}")
                
                # Add context to agent's system prompt
                event.agent.system_prompt += f"\n\nRecent conversation history:\n{context}\n\nContinue the conversation naturally based on this context."
                
                logger.info(f"✅ Loaded {len(recent_turns)} recent conversation turns")
            else:
                logger.info("No previous conversation history found")
                
        except Exception as e:
            logger.error(f"Failed to load conversation history: {e}")
    
    def on_message_added(self, event):
        """Store conversation turns in memory"""
        try:
            print(f"message added ------- {event.message}")
            self.memory_client.save_conversation(
                memory_id=self.memory_id,
                actor_id=self.actor_id,
                session_id=self.session_id,
                messages=[(event.message["content"][0]["text"], event.message["role"])]
            )
            
        except Exception as e:
            logger.error(f"Failed to store message: {e}")
    
    def register_hooks(self, registry: HookRegistry) -> None:
        # Create and register memory hook
        registry.add_callback(MessageAddedEvent, self.on_message_added)
        registry.add_callback(AgentInitializedEvent, self.on_agent_initialized)

## Setting Up Multi-Agent Architecture with Strands Agents
In this section, we'll create our multi-agent system with specialized agents for flight and hotel bookings, both sharing access to our memory resource.

In [40]:
# Create unique actor IDs for each specialized agent but share the session ID
flight_actor_id = f"flight-user-{datetime.now().strftime('%Y%m%d%H%M%S')}"
hotel_actor_id = f"hotel-user-{datetime.now().strftime('%Y%m%d%H%M%S')}"
session_id = f"travel-session-{datetime.now().strftime('%Y%m%d%H%M%S')}"
flight_namespace = f"travel/{flight_actor_id}/preferences"
hotel_namespace = f"travel/{hotel_actor_id}/preferences"

In [41]:
# Import the necessary components
from strands import Agent, tool

### Creating Specialized Agents with Memory Access

Next, we'll define system prompts for our specialized agents. Each prompt includes the memory parameters in a format that the agent can parse:

In [42]:
# System prompt for the hotel booking specialist
HOTEL_BOOKING_PROMPT = f"""You are a hotel booking assistant. Help customers find hotels, make reservations, and answer questions about accommodations and amenities. 
Provide clear information about availability, pricing, and booking procedures in a friendly, helpful manner."""

# System prompt for the flight booking specialist
FLIGHT_BOOKING_PROMPT = f"""You are a flight booking assistant. Help customers find flights, make reservations, and answer questions about airlines, routes, and travel policies. 
Provide clear information about flight availability, pricing, schedules, and booking procedures in a friendly, helpful manner."""

### Implementing Agent Tools
Now we'll implement our specialized agents as tools that can be used by the coordinator agent:

In [43]:
@tool
def flight_booking_assistant(query: str) -> str:
    """
    Process and respond to flight booking queries.

    Args:
        query: A flight-related question about bookings, schedules, airlines, or travel policies

    Returns:
        Detailed flight information, booking options, or travel advice
    """
    try:
        flight_memory_hooks = ShortTermMemoryHook(
            memory_id=memory_id,
            client=client,
            actor_id=flight_actor_id,
            session_id=session_id
        )
        
        flight_agent = Agent(hooks=[flight_memory_hooks], system_prompt=FLIGHT_BOOKING_PROMPT)

        # Call the agent and return its response
        response = flight_agent(query)
        return str(response)
    except Exception as e:
        return f"Error in flight booking assistant: {str(e)}"

@tool
def hotel_booking_assistant(query: str) -> str:
    """
    Process and respond to hotel booking queries.

    Args:
        query: A hotel-related question about accommodations, amenities, or reservations

    Returns:
        Detailed hotel information, booking options, or accommodation advice
    """
    try:
        hotel_memory_hooks = ShortTermMemoryHook(
            memory_id=memory_id,
            client=client,
            actor_id=hotel_actor_id,
            session_id=session_id
        )

        hotel_booking_agent = Agent(hooks=[hotel_memory_hooks], system_prompt=HOTEL_BOOKING_PROMPT)
        
        # Call the agent and return its response
        response = hotel_booking_agent(query)
        return str(response)
    except Exception as e:
        return f"Error in hotel booking assistant: {str(e)}"

### Creating the Coordinator Agent

Finally, we'll create the main travel planning agent that coordinates between these specialized tools:

In [44]:
# System prompt for the coordinator agent
TRAVEL_AGENT_SYSTEM_PROMPT = """
You are a comprehensive travel planning assistant that coordinates between specialized tools:
- For flight-related queries (bookings, schedules, airlines, routes) → Use the flight_booking_assistant tool
- For hotel-related queries (accommodations, amenities, reservations) → Use the hotel_booking_assistant tool
- For complete travel packages → Use both tools as needed to provide comprehensive information
- For general travel advice or simple travel questions → Answer directly

Each agent will have its own memory in case the user asks about historic data.
When handling complex travel requests, coordinate information from both tools to create a cohesive travel plan.
Provide clear organization when presenting information from multiple sources. \
Ask max two questions per turn. Keep the messages short, don't overwhelm the customer.
"""

In [45]:
travel_agent = Agent(
    system_prompt=TRAVEL_AGENT_SYSTEM_PROMPT,
    tools=[flight_booking_assistant, hotel_booking_assistant]
)

2025-07-14 07:49:24 - INFO - Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole


## Testing the Multi-Agent Memory System

Let's test our multi-agent system with a travel planning scenario:

In [46]:
response = travel_agent("Hello, I would like to book a trip from LA to Madrid. From July 1 to August 2.")

Hello! I'd be happy to help you plan your trip from Los Angeles to Madrid from July 1 to August 2. Let's work on both your flight and hotel arrangements.

First, let me help you with flight options:
Tool #1: flight_booking_assistant
And for hotel accommodations during your stay:
Tool #2: hotel_booking_assistant
I apologize for the technical difficulties with our booking tools. Let me help you directly with planning your trip:

For your flight from LA to Madrid (July 1 to August 2):
- This is a long-haul international flight (approximately 10-12 hours)
- Airlines that typically serve this route include Iberia, American Airlines, Delta, and Air Europa
- For best prices, I recommend booking at least 2-3 months in advance
- Consider if you prefer direct flights or are open to connections

For your accommodation in Madrid:
- Madrid offers various accommodation options from luxury hotels to budget-friendly hostels
- Popular areas to stay include Sol, Gran Vía, Salamanca, and Malasaña
- A mon

In [38]:
response = travel_agent("I would only like to focus on the flight at the moment. direct flimid-range, city center, pool, standard room")

I understand you'd like to focus on flights at the moment. Let me help you find direct flight options from LA to Madrid for your July 1 to August 2 trip.
Tool #4: flight_booking_assistant
I apologize for the continued technical difficulties with our flight booking tool. Based on your request for direct flights, here's some general information:

For direct flights from LA to Madrid (July 1 - August 2):

1. Airlines offering direct routes: 
   - Iberia (Spain's flag carrier)
   - American Airlines (codeshare with Iberia)

2. Flight duration: Approximately 11-12 hours eastbound (LA to Madrid)

3. Departure airport: Los Angeles International Airport (LAX)

4. Arrival airport: Adolfo Suárez Madrid-Barajas Airport (MAD)

5. Typical departure times: Evening departures from LAX (around 5-7 PM), arriving in Madrid the following afternoon

6. Price range: For July-August (peak season), direct flights typically range from $900-1,500 for economy class

Would you like me to provide any specific inf

## Testing Memory Persistence

To test if our memory system is working correctly, we'll create a new instance of the travel agent and see if it can access the previously stored information:

In [47]:
# Create a new instance of the travel agent
new_travel_agent = Agent(
    system_prompt=TRAVEL_AGENT_SYSTEM_PROMPT,
    tools=[flight_booking_assistant, hotel_booking_assistant]
)

# Ask about previous conversations
new_travel_agent("Can you remind me about flights talked about before?")

2025-07-14 07:49:46 - INFO - Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole


I'd be happy to check your previous flight discussions. Let me access that information for you.
Tool #1: flight_booking_assistant
I apologize, but I'm unable to retrieve your previous flight discussions at the moment due to a technical issue with our system. 

Would you mind sharing some details about the flights you're trying to recall? For example:
- When did you discuss these flights?
- Were they for a specific destination?
- Were they related to an upcoming trip?

AgentResult(stop_reason='end_turn', message={'role': 'assistant', 'content': [{'text': "I apologize, but I'm unable to retrieve your previous flight discussions at the moment due to a technical issue with our system. \n\nWould you mind sharing some details about the flights you're trying to recall? For example:\n- When did you discuss these flights?\n- Were they for a specific destination?\n- Were they related to an upcoming trip?"}]}, metrics=EventLoopMetrics(cycle_count=2, tool_metrics={'flight_booking_assistant': ToolMetrics(tool={'toolUseId': 'tooluse_DyTOEi7ZRhu5CkoDtucEwQ', 'name': 'flight_booking_assistant', 'input': {'query': 'Show previous flight discussions or bookings'}}, call_count=1, success_count=1, error_count=0, total_time=0.0004215240478515625)}, cycle_durations=[3.3178231716156006], traces=[<strands.telemetry.metrics.Trace object at 0x7f582b84b850>, <strands.telemetry.metrics.Trace object at 0x7f582b84bc70>], accumulated_usage={'inputTokens': 1608, 'outputTokens': 159

## Summary

In this notebook, we've demonstrated:

1. How to create a shared memory resource for multiple agents
2. How to implement specialized agents as tools with memory access
3. How to coordinate between multiple agents while maintaining conversation context
4. How memory persists across different agent instances

This multi-agent architecture with shared memory provides a powerful approach for building complex conversational AI systems that can handle specialized domains while maintaining a cohesive user experience.

## Clean up
Let's delete the memory to clean up the resources used in this notebook.

In [ ]:
client.delete_memory_and_wait(
        memory_id = memory_id,
        max_wait = 300,
        poll_interval =10
)